In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# 初始化 Mediapipe 姿勢偵測
mp_pose = mp.solutions.pose

# 計算三點之間角度的函數
def calculate_angle(a, b, c):
    a = np.array(a)  # 第一點
    b = np.array(b)  # 中間點
    c = np.array(c)  # 第三點

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

# 處理視頻檔案
cap = cv2.VideoCapture('./video/test5.mp4')

# 姿勢偵測
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 轉換顏色空間並處理圖像
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # 提取關節點座標並計算角度
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            shoulder = [landmarks[11].x, landmarks[11].y]
            elbow = [landmarks[13].x, landmarks[13].y]
            wrist = [landmarks[15].x, landmarks[15].y]

            # 繪製點和連線
            for landmark in [shoulder, elbow, wrist]:
                x, y = int(landmark[0] * frame.shape[1]), int(landmark[1] * frame.shape[0])
                cv2.circle(image, (x, y), 5, (255, 0, 0), -1)

            cv2.line(image, (int(shoulder[0] * frame.shape[1]), int(shoulder[1] * frame.shape[0])),
                           (int(elbow[0] * frame.shape[1]), int(elbow[1] * frame.shape[0])), (255, 0, 0), 2)
            cv2.line(image, (int(elbow[0] * frame.shape[1]), int(elbow[1] * frame.shape[0])),
                           (int(wrist[0] * frame.shape[1]), int(wrist[1] * frame.shape[0])), (255, 0, 0), 2)

            angle = calculate_angle(shoulder, elbow, wrist)

            # 顯示角度
            cv2.putText(image, str(int(angle)),
                        tuple(np.multiply(elbow, [frame.shape[1], frame.shape[0]]).astype(int)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

        # 顯示處理後的影像
        cv2.imshow('MediaPipe Pose', image)

        # 按下 'ESC' 鍵退出
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

ModuleNotFoundError: ignored